In [1]:
import sys
from pathlib import Path
path = Path.cwd().parent.resolve().as_posix()
if path not in sys.path:
    sys.path.append(path)

In [2]:
import numpy as np
import statsmodels.formula.api as smf
from numpy import e, log, dot, array, matrix, ones
from numpy.random import normal, binomial
from statsmodels.api import families
from graphviz import Digraph

C:\Users\petalp\Applications\Anaconda\envs\simulation-presentation\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Simulation Part 6 - Generalized Linear Models

- $E(y) = \mu = g^{-1}(\boldsymbol{X}\boldsymbol{\beta})$
- $y = distribution(\mu)$

## Let's define some link functions and their inverse functions

In [3]:
def identity(x):
    return x

In [4]:
def inv_identity(y):
    return y

In [5]:
def logit(x):
    return log(x / (1 - x))

In [6]:
def inv_logit(y):
    return e ** y / (1 + e ** y)

## Some helper functions

In [7]:
def binomial_1(p):
    return binomial(1, p)

In [8]:
def design_matrix(xs):
    # Turn into matrix
    xs = array(xs).T
    # Return with intercept column
    return np.c_[ones(xs.shape[0]), xs]

In [9]:
def generate_outcome(X, B, inv_link_func, distribution):
    # Calculate dot product
    XB = dot(X, B)
    # Get estimated value for each obs
    E = inv_link_func(XB)
    # Return values from the distribution
    return distribution(E)

## Generate our data

In [10]:
n = 1000

In [11]:
X = design_matrix([
    normal(size=n),
    normal(size=n)])
X

array([[ 1.        ,  1.40171879,  2.07622498],
       [ 1.        , -0.48136743, -1.0689299 ],
       [ 1.        , -0.10924019,  1.37424603],
       ...,
       [ 1.        ,  0.822403  , -0.13585403],
       [ 1.        , -0.05145067,  0.13178135],
       [ 1.        ,  0.27001949, -0.64957169]])

In [12]:
B = [1, 2, 3]

In [13]:
y_linear = generate_outcome(X, B, inv_identity, normal)

In [14]:
y_logit = generate_outcome(X, B, inv_logit, binomial_1)

## Fit our data

In [15]:
linear_family = families.Gaussian(families.links.identity)
smf.GLM(y_linear, X, family=linear_family).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                 1000
Model:                            GLM   Df Residuals:                      997
Model Family:                Gaussian   Df Model:                            2
Link Function:               identity   Scale:              0.9853998479404538
Method:                          IRLS   Log-Likelihood:                -1410.1
Date:                Mon, 28 May 2018   Deviance:                       982.44
Time:                        08:54:51   Pearson chi2:                     982.
No. Iterations:                     2                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9924      0.031     31.553      0.000       0.931       1.054
x1             2.0425      0.030     67.810      0.000       1.983       2.102
x2             3.0186      0.030     99.833      0.000       2.959       3.078
==============================================================================
"""

In [16]:
logreg_family = families.Binomial(families.links.logit)
smf.GLM(y_logit, X, family=logreg_family).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                 1000
Model:                            GLM   Df Residuals:                      997
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -277.38
Date:                Mon, 28 May 2018   Deviance:                       554.76
Time:                        08:54:51   Pearson chi2:                     700.
No. Iterations:                     7                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0109      0.119      8.486      0.000       0.777       1.244
x1             2.2501      0.171     13.145      0.000       1.915       2.586
x2             3.0597      0.215     14.228      0.000       2.638       3.481
==============================================================================
"""